# 네이버 뷰탭 제목 크롤링

In [48]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

In [14]:
import selenium
from selenium import webdriver

path = 'chromedriver.exe'
driver = webdriver.Chrome(path)

In [51]:
search = "농협양곡"
text = quote(search)

In [52]:
driver.get(f'https://search.naver.com/search.naver?where=view&sm=tab_jum&query={text}')

In [53]:
title_list = []

1차 에러 -> try except으로 처리   
2차 에러 -> ?

In [54]:
import time

for i in range(1, 1000):
    try : 
        a = driver.find_element_by_css_selector(f'#main_pack > section > div > div._list > panel-list > div:nth-child(1) > more-contents > div > ul > li:nth-child({i}) > div.total_wrap.api_ani_send > div > a').text
        title_list.append(a)
    except : 
        SCROLL_PAUSE_SEC = 1

        # 스크롤 높이 가져옴
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # 끝까지 스크롤 다운
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # 1초 대기
            time.sleep(SCROLL_PAUSE_SEC)

            # 스크롤 다운 후 스크롤 높이 다시 가져옴
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        time.sleep(2)
        print('스크롤 내림')
        a = driver.find_element_by_css_selector(f'#main_pack > section > div > div._list > panel-list > div:nth-child(1) > more-contents > div > ul > li:nth-child({i}) > div.total_wrap.api_ani_send > div > a').text
        title_list.append(a)

스크롤 내림
스크롤 내림


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#main_pack > section > div > div._list > panel-list > div:nth-child(1) > more-contents > div > ul > li:nth-child(124) > div.total_wrap.api_ani_send > div > a"}
  (Session info: chrome=92.0.4515.107)


In [55]:
len(title_list)

123

In [56]:
import pandas as pd

df = pd.DataFrame({'title_list' : title_list})
df.head()

,title_list
0,#안성시 #미양농협 #양곡유통센터 #화재
1,농협안성 양곡 유통센터 화재
2,"[자율활동/강소기업 일일 기자단 인터뷰]3.5단계. 쌀 유통 경로 조사 """" 인터뷰"
3,허름한 과거 잊어줘요~카페·미술관…이젠 핫플!지역명소로 뜬 ‘농협 폐창고’
4,⭕ 경기 안성 미양면 농협 양곡유통센터 화재 불 ! 직원 1명 화상 시청 인근 주민...


In [57]:
df.to_csv(f'네이버_블로그제목_{len(title_list)}개_{search}.csv',encoding='utf-8-sig')